# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [36]:
import os
import glob
import psycopg2
import pandas as pd
import json

In [37]:
from sql_queries import *

In [38]:
with open("../config.json") as f:
    config = json.load(f)
    
conn = psycopg2.connect(host = config["host"],
                        dbname = config["dbname"],
                        user = config["username"],
                        password = config["password"])

conn.set_session(autocommit=True)
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

## Due to our Foreign key constraints, we will have to fill parent tables first, before filling child tables.
For that, we can fill the tables in the following order:
1. artist table (dimension table)
2. Song table (dimension table)
3. User table (dimension table)
4. Time Table (dimension table)
5. song played table (fact table)

## Process Artist Data

### Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files("../data/song_data/")
song_files[:5]

['c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json',
 'c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json',
 'c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json',
 'c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json',
 'c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json']

In [7]:
filepath = song_files[0]

In [8]:
df = pd.read_json(filepath, lines = True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


### Artist table

In [9]:
columns = ["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]
artist_data = df.loc[0, columns]
artist_data = list(artist_data.values)
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [10]:
cur.execute(artist_table_insert, artist_data)

UniqueViolation: duplicate key value violates unique constraint "artist_pkey"
DETAIL:  Key (artist_id)=(ARD7TVE1187B99BFB1) already exists.


We need a way to map numpy nans to Nulls in the database. Using Pycopg2, we can do the following

In [11]:
import numpy as np
from psycopg2.extensions import AsIs, register_adapter

def adapt_npfloat_and_nans(np_float,
                           _FLOAT = psycopg2.extensions.Float,
                           _NULL = psycopg2.extensions.AsIs('Null')):
    if np.isnan(np_float):
        return _NULL
    return _FLOAT(np_float)

register_adapter(np.float64, adapt_npfloat_and_nans)

We can do a sanity check to see if it works

In [12]:
cur.execute("SELECT * FROM artist")
res = cur.fetchone()
while res:
    print(res)
    k = res
    res = cur.fetchone()

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan)


## #2: `songs` Table

In [13]:
columns = ["song_id", "title", "artist_id", "year", "duration"]
song_data = df.loc[0, columns]
song_data = list(song_data.values)
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
cur.execute(song_table_insert, song_data)

ProgrammingError: can't adapt type 'numpy.int64'

This is an error that occurs because the database driver doesn't know what to do with np datatypes. A proposed solution on [stackoverflow](https://stackoverflow.com/questions/50626058/psycopg2-cant-adapt-type-numpy-int64) shows that we can use the string representation of those datatypes as proxy for their value, because their string representation shouldn't differ from their value.

This is done via the `AsIs` extension method in psycopg2, and regsitering the string representation such that the driver does that representation whenever it encounters a value of that type

Another solution [Stackoverflow](https://stackoverflow.com/questions/32107558/how-do-i-convert-numpy-nan-objects-to-sql-nulls)

In [15]:
import numpy as np
from psycopg2.extensions import AsIs, register_adapter
def adapt_np_int(np_int:np.int64):
    _INT = psycopg2.extensions.Int
    return _INT(np_int)
    
register_adapter(np.int64, adapt_np_int)

In [16]:
cur.execute(song_table_insert, song_data)

UniqueViolation: duplicate key value violates unique constraint "song_pkey"
DETAIL:  Key (song_id)=(SOMZWCG12A8C13C480) already exists.


Now for the sanity check

In [17]:
cur.execute("SELECT * FROM song")
res = cur.fetchone()
print(res)

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
log_files = get_files("../data/log_data")

In [19]:
filepath = log_files[0]

In [20]:
df_log = pd.read_json(filepath, lines=True)
df_log.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
df_songplay = df_log[df_log["page"] == "NextSong"].copy().reset_index()
df_songplay.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
1,4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [22]:
df_songplay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          11 non-null     int64  
 1   artist         11 non-null     object 
 2   auth           11 non-null     object 
 3   firstName      11 non-null     object 
 4   gender         11 non-null     object 
 5   itemInSession  11 non-null     int64  
 6   lastName       11 non-null     object 
 7   length         11 non-null     float64
 8   level          11 non-null     object 
 9   location       11 non-null     object 
 10  method         11 non-null     object 
 11  page           11 non-null     object 
 12  registration   11 non-null     int64  
 13  sessionId      11 non-null     int64  
 14  song           11 non-null     object 
 15  status         11 non-null     int64  
 16  ts             11 non-null     int64  
 17  userAgent      11 non-null     object 
 18  userId      

In [23]:
session_datetime = pd.to_datetime(df_songplay["ts"], unit = "ms")
session_datetime

0    2018-11-01 21:01:46.796
1    2018-11-01 21:05:52.796
2    2018-11-01 21:08:16.796
3    2018-11-01 21:11:13.796
4    2018-11-01 21:17:33.796
5    2018-11-01 21:24:53.796
6    2018-11-01 21:28:54.796
7    2018-11-01 21:42:00.796
8    2018-11-01 21:52:05.796
9    2018-11-01 21:55:25.796
10   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns]

In [24]:
ts = df_songplay["ts"]
hour = session_datetime.dt.hour
day = session_datetime.dt.day
weekday = session_datetime.dt.weekday
week = session_datetime.dt.isocalendar().week
month = session_datetime.dt.month
year = session_datetime.dt.year

In [25]:
#time_data = (ts, hour, day, weekday, week, month, year)
#column_labels = ("ts", "hour", "day", "weekday", "week", "month", "year")
time_data_dict = {
    "ts": ts,
    "hour":hour,
    "day": day,
    "weekday": weekday,
    "week": week,
    "month": month,
    "year": year
}

In [26]:
time_df = pd.DataFrame(time_data_dict)
time_df.head()

,ts,hour,day,weekday,week,month,year
0,1541106106796,21,1,3,44,11,2018
1,1541106352796,21,1,3,44,11,2018
2,1541106496796,21,1,3,44,11,2018
3,1541106673796,21,1,3,44,11,2018
4,1541107053796,21,1,3,44,11,2018


In [27]:
cur.execute("""
                     CREATE TABLE IF NOT EXISTS time_table(
                         start_time BIGINT,
                         hour INT,
                         day INT,
                         weekday INT,
                         week INT,
                         month INT,
                         year INT
                     );
""")

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [28]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    #conn.commit()

In [29]:
cur.execute("SELECT * FROM time_table")
res = cur.fetchone()
while(res):
    print(res)
    res = cur.fetchone()

(1541106106796, 21, 1, 3, 44, 11, 2018)
(1541106352796, 21, 1, 3, 44, 11, 2018)
(1541106496796, 21, 1, 3, 44, 11, 2018)
(1541106673796, 21, 1, 3, 44, 11, 2018)
(1541107053796, 21, 1, 3, 44, 11, 2018)
(1541107493796, 21, 1, 3, 44, 11, 2018)
(1541107734796, 21, 1, 3, 44, 11, 2018)
(1541108520796, 21, 1, 3, 44, 11, 2018)
(1541109125796, 21, 1, 3, 44, 11, 2018)
(1541109325796, 21, 1, 3, 44, 11, 2018)
(1541110994796, 22, 1, 3, 44, 11, 2018)
(1541106106796, 21, 1, 3, 44, 11, 2018)
(1541106352796, 21, 1, 3, 44, 11, 2018)
(1541106496796, 21, 1, 3, 44, 11, 2018)
(1541106673796, 21, 1, 3, 44, 11, 2018)
(1541107053796, 21, 1, 3, 44, 11, 2018)
(1541107493796, 21, 1, 3, 44, 11, 2018)
(1541107734796, 21, 1, 3, 44, 11, 2018)
(1541108520796, 21, 1, 3, 44, 11, 2018)
(1541109125796, 21, 1, 3, 44, 11, 2018)
(1541109325796, 21, 1, 3, 44, 11, 2018)
(1541110994796, 22, 1, 3, 44, 11, 2018)


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`


In [30]:
df_log.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [31]:
df_users = df_log.copy()
#df_users.set_index("userId", inplace = True)
#df_users.head(2)

In [32]:
columns = ["userId", "firstName", "lastName", "gender", "level"]
df_users = df_users[columns]
df_users.head()

,userId,firstName,lastName,gender,level
0,39,Walter,Frye,M,free
1,8,Kaylee,Summers,F,free
2,8,Kaylee,Summers,F,free
3,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free


We should note that our user data can change. It makes sense to make them able to switch between free and premium levels. For the other data, some platforms allow this data to change, other do not, so it's up to use to decide. I will go with enabling the users to change their other information as well.


We can opt to do an update query before inserting, in order to avoid primary key conflict. However, we can do an insertion and on-conflict, update the other information, in other words, do an "upsert" operation

As such, we should take care of the way, and order of data insertion.
* On the level of files, we need to upsert data in the correct order, that way, we don't override newer data with older data.

* Within each file, we need to sort the data by timestamp, to satisfy the same constraint.

* To minimize the number of upsert operations and reduce the number of requests to the database, we can read all the logs data in a single dataframe, group them by user, and pick the last record of that user (in terms of timestamp) as the final value to be inserted into our database. If we can't read all the data into memory, we can do that file by file.

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [39]:
for i, row in df_users.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [42]:
cur.execute("SELECT * FROM user_table")
res = cur.fetchall()
res

[('39', 'Walter', 'Frye', 'M', 'free'),
 ('8', 'Kaylee', 'Summers', 'F', 'free'),
 ('10', 'Sylvie', 'Cruz', 'F', 'free'),
 ('26', 'Ryan', 'Smith', 'M', 'free'),
 ('101', 'Jayden', 'Fox', 'M', 'free')]

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [35]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.